In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
import matplotlib.pyplot as plt
import numpy as np
import re, os
import seaborn as sns
import nltk
from string import printable
from sklearn import model_selection
from sklearn.metrics import accuracy_score, classification_report
from matplotlib import pyplot
from pathlib import Path
import json
import warnings
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
warnings.filterwarnings("ignore")



In [ ]:
# Reading the Finale_Labeled_Dataset
Al_Boghdady_Multi_Class_Dataset = pd.read_csv('Al_Boghdady_Multi_Class.csv',encoding = 'unicode_escape')

In [ ]:
# Display the first few rows of the dataset
print("First few rows of the dataset:")
print(Al_Boghdady_Multi_Class_Dataset.head())

First few rows of the dataset:
                                                code isMalicious
0  void printfUART_buf  ( char *buf ,   int  len ...     CWE-561
1  check_opt_size  ( cp_opt_t *opt ,  unsigned ch...     CWE-561
2  cp_ft  ( cp_queue_t *queue ,  cp_tid_t id )   ...     CWE-561
3  start  ( cp_pdu_t *pdu )   {  if   ( pdu && pd...     CWE-561
4  cp_clone_pdu  ( cp_pdu_t *pdu )   {  cp_pdu_t ...     CWE-561


In [ ]:
# Display information about the dataset (e.g., column names, data types)
print("\nInformation about the dataset:")
print(Al_Boghdady_Multi_Class_Dataset.info())


Information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         5117 non-null   object
 1   isMalicious  5117 non-null   object
dtypes: object(2)
memory usage: 80.1+ KB
None


In [ ]:
# Check for missing values
print("\nMissing values in the dataset:")
print(Al_Boghdady_Multi_Class_Dataset.isnull().sum())


Missing values in the dataset:
code           0
isMalicious    0
dtype: int64


In [ ]:
import plotly.express as px

# Create a DataFrame for missing values
missing_values_df = Al_Boghdady_Multi_Class_Dataset.isnull().sum().reset_index()
missing_values_df.columns = ['Column', 'Missing Values']

# Create an interactive bar plot
fig = px.bar(missing_values_df, x='Column', y='Missing Values',
             title='Missing Values in the Dataset',
             labels={'Missing Values': 'Number of Missing Values', 'Column': 'Column Name'})

# Customize the layout
fig.update_layout(xaxis_title='Column Name', yaxis_title='Number of Missing Values')

# Show the interactive plot
fig.show()

In [ ]:
# Check the distribution of the target variable
print("\nDistribution of the target variable:")
print(Al_Boghdady_Multi_Class_Dataset['isMalicious'].value_counts())


Distribution of the target variable:
Benign Code    1953
CWE-561         237
CWE-398         168
CWE-119         117
CWE-476          77
CWE-762          75
CWE-807          74
CWE-477          74
CWE-686          71
CWE-672          71
CWE-685          70
CWE-563          69
CWE-190          68
CWE-367          67
CWE-570          65
CWE-126          65
CWE-401          65
CWE-457          61
CWE-475          60
CWE-664          58
CWE-362          58
CWE-120          58
CWE-783          57
CWE-369          57
CWE-20           56
CWE-467          56
CWE-665          55
CWE-786          54
CWE-758          54
CWE-415          52
CWE-676          52
CWE-377          51
CWE-704          51
CWE-78           50
CWE-687          48
CWE-682          48
CWE-327          48
CWE-134          47
CWE-628          47
CWE-768          46
CWE-571          46
CWE-252          46
CWE-244          43
CWE-732          40
CWE-562          37
CWE-788          36
CWE-683          31
CWE-350          31
CW

In [ ]:
import plotly.express as px

# Check the distribution of the target variable
target_distribution = Al_Boghdady_Multi_Class_Dataset['isMalicious'].value_counts().reset_index()
target_distribution.columns = ['Class', 'Count']

# Create an interactive bar plot
fig = px.bar(target_distribution, x='Class', y='Count',
             title='Distribution of the Target Variable',
             labels={'Count': 'Number of Instances', 'Class': 'Malicious Class'},
             color='Class')

# Customize the layout
fig.update_layout(xaxis_title='Malicious Class', yaxis_title='Number of Instances')

# Show the interactive plot
fig.show()

In [ ]:
Al_Boghdady_Multi_Class_Dataset.shape

(5117, 2)

In [ ]:
# Checking for duplicate rows or null values
Al_Boghdady_Multi_Class_Dataset.dropna(inplace=True)
Al_Boghdady_Multi_Class_Dataset.drop_duplicates(inplace=True)
Al_Boghdady_Multi_Class_Dataset.shape

(4810, 2)

# Preprocessing for Machine Learning Classification

In the realm of machine learning, classifiers often require input in the form of fixed-length feature vectors. When dealing with unstructured textual data, a common challenge arises: how do we convert this unstructured information into a numerical array that is meaningful for machine learning algorithms?

The process of converting unstructured text into a format suitable for machine learning is known as preprocessing. This critical step involves transforming raw textual data into a structured and numerical representation that machine learning models can comprehend and learn from.

In our specific context, where the goal is to detect vulnerabilities in IoT operating system source code, preprocessing plays a pivotal role. We are dealing with code snippets written in C/C++, and it's essential to convert this code into a format that can be fed into machine learning classifiers. This ensures that the algorithms can effectively identify patterns and make informed predictions.

The subsequent sections will delve into the preprocessing steps undertaken to convert the raw textual data of IoT operating system source code into a feature vector of fixed length. This transformation is integral to the success of our machine learning models, ultimately contributing to the accurate detection of vulnerabilities in the codebase.

Understanding and implementing preprocessing steps allow us to bridge the gap between the inherent structure of code and the requirements of machine learning algorithms, facilitating a seamless integration of textual data into the classification pipeline.

## Tokenization
Tokenization serves as the foundational step in the process of transforming our data into a format suitable for analysis. Particularly crucial in Natural Language Processing (NLP) projects, tokenization involves breaking down each textual "document," in this context, code snippets, into individual character substrings known as tokens.

The tokens extracted from the code snippets lay the groundwork for constructing a set of words, commonly referred to as a vocabulary. This vocabulary becomes instrumental in the subsequent vectorization of the text, a process essential for many NLP tasks. While tokenization for regular English text often involves simple separation of words with space characters and punctuation, this approach may have limitations.

For datasets like code snippets, advanced tokenization techniques are necessary. Packages such as nltk or spacy offer sophisticated tokenizers that are adept at handling code-related structures and nuances. Implementing a robust tokenization strategy is pivotal for generating a meaningful and comprehensive vocabulary, laying the groundwork for subsequent data analysis and machine learning tasks.

##Data Split


The dataset is split into training and testing sets, with code snippets stored in the variable Code_Snippet and corresponding tags (malicious or benign) in Code_Tag for further analysis and model evaluation.

In [ ]:
# Dataset split for training and testing.
Code_Snippet, Code_Tag = Al_Boghdady_Multi_Class_Dataset.code, Al_Boghdady_Multi_Class_Dataset.isMalicious

##Cleanup
In the cleanup phase, eliminate single-character variable names or sequences of the same character (e.g., xxx) for enhanced code readability and maintainability.

In [ ]:
# Cleaning-up
def preprocess(Code_Snippet):
    return pd.Series(Code_Snippet).replace(r'\b([A-Za-z])\1+\b', '', regex=True)\
        .replace(r'\b[A-Za-z]\b', '', regex=True)
transformer = FunctionTransformer(preprocess)

##Term Frequency - Inverse Document Frequency

In the context of code analysis, traditional NLP steps like lemmatization or stemming may lead to information loss, given that code is not a "natural" language. To preserve the distinctive structure of programming languages and pinpoint key discriminative keywords, we skip lemmatization/stemming and directly proceed to vectorization.

Our chosen vectorization method is TF-IDF (term frequency-inverse document frequency), which represents each source code document with an M-sized array. Each element in the array corresponds to the scaled frequency of a token in the document. The TF-IDF factor for a token (i) in a document (d) is computed using the formula:

###TF-IDF(i,d)=TF(i,d)×IDF(i)###

Here, TF denotes term frequency, and IDF is the inverse document frequency. The IDF factor aims to diminish the weight of tokens that appear frequently across all documents (e.g., common operators like '='), providing a scaled representation that emphasizes tokens with higher information content about the document's nature.

In [ ]:
token_pattern = r"""([A-Za-z_]\w*\b|[!\#\$%\&\*\+:\-\./<=>\?@\\\^_\|\~]+|[ \t\(\),;\{\}\[\]"'`])"""
vectorizer = TfidfVectorizer(token_pattern=token_pattern, max_features=3000)
Code_Snippet_train, Code_Snippet_test, Code_Tag_train, Code_Tag_test = train_test_split(Code_Snippet, Code_Tag, test_size=0.15, shuffle=True)


In the above cell, a custom token pattern is defined using a regular expression (token_pattern) to capture relevant elements in the source code. The TF-IDF vectorization is then applied using Scikit-learn's TfidfVectorizer. Additionally, the parameter max_features is employed to restrict the features to the top 3000 tokens based on their term frequency across the corpus. This strategic use of max_features aims to enhance computational efficiency, mitigate overfitting, and streamline the feature set for improved model performance. The dataset is subsequently split into training and testing sets using train_test_split.

#Classification using Gradient Boosting:

Here, we leverage the power of Gradient Boosting, a robust ensemble learning technique, for the classification of source code. A pipeline comprising preprocessing, TF-IDF vectorization, and a Gradient Boosting Classifier is established to effectively capture and classify the distinctive features of code snippets.

Utilizing a Gradient Boosting Classifier, our approach to source code classification involves a comprehensive pipeline encompassing preprocessing, TF-IDF vectorization, and the classifier itself. The integration of optimal hyperparameters, such as the number of estimators, learning rate, and maximum depth, has been meticulously tuned for enhanced model accuracy.

After setting the parameters, the pipeline is fitted to the training data (Code_Snippet_train, Code_Tag_train). The accuracy of the trained model is then evaluated on the testing set (Code_Snippet_test, Code_Tag_test), yielding an accuracy score of approximately 94.6%. This result indicates the model's effectiveness in correctly classifying source code snippets into their respective categories.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Defining the Training Model Classifier for Multi-Class Classification
clf_dt = DecisionTreeClassifier()

pipe_DT = Pipeline([
   ('preprocessing', transformer),
   ('vectorizer', vectorizer),
   ('clf', clf_dt)
])

# Setting of the best parameters (for Decision Tree)
best_params_dt = {
    'clf__criterion': 'gini',
    'clf__splitter': 'best',
    'clf__min_samples_split': 3,
    'clf__max_features': 'sqrt'
}

pipe_DT.set_params(**best_params_dt)

# Fitting
pipe_DT.fit(Code_Snippet_train, Code_Tag_train)

# Evaluation of the training model
print(f'Accuracy: {pipe_DT.score(Code_Snippet_test, Code_Tag_test)}')

Accuracy: 0.871191135734072


In [ ]:
pred = pipe_DT.predict(Code_Snippet_test)
pred

array(['CWE-477', 'Benign Code', 'Benign Code', 'CWE-561', 'CWE-762',
       'CWE-367', 'CWE-120', 'Benign Code', 'CWE-563', 'Benign Code',
       'CWE-415', 'CWE-762', 'Benign Code', 'CWE-807', 'CWE-758',
       'CWE-20', 'CWE-775', 'Benign Code', 'CWE-686', 'CWE-190',
       'CWE-561', 'CWE-362', 'Benign Code', 'CWE-467', 'Benign Code',
       'CWE-665', 'CWE-369', 'CWE-377', 'CWE-401', 'CWE-665',
       'Benign Code', 'CWE-119', 'Benign Code', 'CWE-120', 'Benign Code',
       'Benign Code', 'CWE-570', 'Benign Code', 'CWE-190', 'CWE-685',
       'Benign Code', 'CWE-457', 'Benign Code', 'Benign Code', 'CWE-78',
       'CWE-762', 'CWE-563', 'CWE-398', 'CWE-758', 'Benign Code',
       'Benign Code', 'CWE-686', 'Benign Code', 'CWE-685', 'Benign Code',
       'CWE-252', 'CWE-571', 'Benign Code', 'Benign Code', 'Benign Code',
       'Benign Code', 'CWE-119', 'CWE-562', 'CWE-377', 'CWE-664',
       'CWE-252', 'Benign Code', 'Benign Code', 'CWE-628', 'Benign Code',
       'Benign Code', 'Ben

In [ ]:
Al_Boghdady_Multi_Class_Dataset.shape
Code_Snippet_Features = Al_Boghdady_Multi_Class_Dataset.code
Code_Snippet_Tags = Al_Boghdady_Multi_Class_Dataset.isMalicious

Below, we utilized a Decision Tree Classifier for multi-class classification, employing a K-Fold cross-validator with 20 splits to assess the model's performance across different subsets of the dataset. Each iteration involved training the classifier on one set and evaluating its accuracy on another. The results showcase the accuracy achieved in each fold, demonstrating the model's consistency and effectiveness.

The individual accuracy scores varied between approximately 85.1% and 92.9%, with a mean accuracy of 88.6%. The visual representation of K-Fold cross-validation results is presented in the plotted graph, providing a comprehensive view of the model's performance across different splits.

These outcomes illustrate the effectiveness of the Decision Tree model in classifying source code snippets. The mean accuracy serves as a reliable indicator of its overall performance, indicating the model's capability in accurately categorizing code into various classes, contributing to the broader goals of the project.

In [ ]:
# Decision Tree Training Model for Multi-Class Classification
Scores_dt = []
x_dt = [i for i in range(1, 21)]
kf = KFold(n_splits=20, random_state=None, shuffle=True)

for train_index, test_index in kf.split(Code_Snippet_Features):
    Code_Snippet_train, Code_Snippet_test = Code_Snippet_Features.iloc[train_index], Code_Snippet_Features.iloc[test_index]
    Code_Tag_train, Code_Tag_test = Code_Snippet_Tags.iloc[train_index], Code_Snippet_Tags.iloc[test_index]

    clf_dt = DecisionTreeClassifier()
    pipe_DT = Pipeline([
        ('preprocessing', transformer),
        ('vectorizer', vectorizer),
        ('clf', clf_dt)
    ])

    # Setting best params
    best_params_dt = {
        'clf__criterion': 'gini',
        'clf__splitter': 'best',
        'clf__min_samples_split': 3,
        'clf__max_features': 'sqrt'
    }

    pipe_DT.set_params(**best_params_dt)

    # Fitting
    pipe_DT.fit(Code_Snippet_train, Code_Tag_train)

    # Evaluation
    print(f'Accuracy: {pipe_DT.score(Code_Snippet_test, Code_Tag_test)}')
    Scores_dt.append(pipe_DT.score(Code_Snippet_test, Code_Tag_test))
    print("-------------------------------------------" )

print("The mean accuracy of Decision Tree training model ", np.mean(Scores_dt))

Accuracy: 0.8547717842323651
-------------------------------------------
Accuracy: 0.8713692946058091
-------------------------------------------
Accuracy: 0.921161825726141
-------------------------------------------
Accuracy: 0.8589211618257261
-------------------------------------------
Accuracy: 0.8921161825726142
-------------------------------------------
Accuracy: 0.8796680497925311
-------------------------------------------
Accuracy: 0.8838174273858921
-------------------------------------------
Accuracy: 0.9294605809128631
-------------------------------------------
Accuracy: 0.9045643153526971
-------------------------------------------
Accuracy: 0.8506224066390041
-------------------------------------------
Accuracy: 0.8666666666666667
-------------------------------------------
Accuracy: 0.8666666666666667
-------------------------------------------
Accuracy: 0.8833333333333333
-------------------------------------------
Accuracy: 0.875
------------------------------------

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_dt, y=Scores_dt, mode='lines+markers', name='Accuracy'))
fig.update_layout(title="The Decision tree training model's K-Folds cross-validator",
                  xaxis_title='The K-Fold',
                  yaxis_title='The accuracy of the Decision Tree training model')

fig.show()

##Feature Extraction

To prepare the unlabeled code snippets for prediction, we utilize the same TF-IDF vectorizer that was employed during the training phase. This ensures consistency in feature representation and allows the model to make predictions based on the learned patterns.

In [ ]:
#DT Prediction Model for Multi-Class Classification
# Reading the TinyOS unlabeled Dataset
TinyOS_unlabeled_Dataset = pd.read_csv('Tinyos_Evaluation.csv',encoding = 'unicode_escape')
TinyOS_Code_Snippet = TinyOS_unlabeled_Dataset.code

def preprocess(TinyOS_Code_Snippet):
    return pd.Series(TinyOS_Code_Snippet).replace(r'\b([A-Za-z])\1+\b', '', regex=True)\
        .replace(r'\b[A-Za-z]\b', '', regex=True)

transformer = FunctionTransformer(preprocess)
token_pattern = r"""([A-Za-z_]\w*\b|[!\#\$%\&\*\+:\-\./<=>\?@\\\^_\|\~]+|[ \t\(\),;\{\}\[\]"'`])"""

vectorizer = TfidfVectorizer(token_pattern=token_pattern, max_features=3000)

* With the preprocessed and vectorized TinyOS code snippets, we proceed to make predictions using the previously trained Decision Tree classifier. The predictions are stored in the 'pred' variable.

* The 'pred' variable now contains the predicted labels for the TinyOS code snippets, allowing us to evaluate the model's performance on unseen data.

In [ ]:
# Print findings
print (TinyOS_Code_Snippet)
pred = pipe_DT.predict(TinyOS_Code_Snippet)
pred

0      void thread ( void* arg )   {  int s ;  for ( ...
1      void thread_main ( void* arg )   {  tosthread_...
2      sf (  )  ;  throttle.initialize (  )  ;  while...
3      int opts ( void *a ,  void *b )   {  if  ( !a ...
4      new_node ( unsigned short key ,  unsigned int ...
                             ...                        
317                                unsigned char keys ; 
318    if  ( BIT_PE ( CS -> CTL0 ,  CS_CTL0_DCORES_OF...
319    else  {  dcoConst  =  * (  ( float * )  &csInf...
320    if  ( BIT_PE ( CS -> CTL0 ,  CS_CTL0_DCORES_OF...
321    void bounceThread1_start ( void *arg )   {  me...
Name: code, Length: 322, dtype: object


array(['Benign Code', 'Benign Code', 'Benign Code', 'Benign Code',
       'Benign Code', 'Benign Code', 'CWE-704', 'CWE-704', 'CWE-704',
       'Benign Code', 'Benign Code', 'CWE-807', 'CWE-682', 'CWE-126',
       'CWE-561', 'CWE-687', 'Benign Code', 'CWE-126', 'CWE-126',
       'CWE-119', 'CWE-686', 'Benign Code', 'CWE-126', 'CWE-126',
       'CWE-768', 'CWE-768', 'Benign Code', 'CWE-807', 'CWE-807',
       'CWE-704', 'CWE-704', 'Benign Code', 'Benign Code', 'CWE-686',
       'CWE-686', 'CWE-686', 'Benign Code', 'Benign Code', 'Benign Code',
       'CWE-571', 'CWE-571', 'CWE-571', 'CWE-704', 'CWE-561', 'CWE-704',
       'CWE-563', 'CWE-563', 'CWE-457', 'CWE-687', 'CWE-686', 'CWE-561',
       'CWE-561', 'CWE-686', 'CWE-686', 'CWE-686', 'CWE-570', 'CWE-570',
       'CWE-561', 'CWE-686', 'CWE-686', 'CWE-686', 'CWE-377', 'CWE-686',
       'CWE-686', 'CWE-686', 'CWE-682', 'CWE-682', 'CWE-561', 'CWE-676',
       'CWE-561', 'CWE-561', 'CWE-561', 'CWE-571', 'CWE-571', 'CWE-570',
       'CWE-5

# Decision Tree Prediction Model Accuracy for iDetect
* After successfully predicting labels for the TinyOS unlabeled dataset using our trained Decision Tree model, it is crucial to evaluate the model's performance. This involves comparing the predicted labels against the ground truth labels from the labeled TinyOS dataset. Below, we assess the accuracy and other relevant metrics for our iDetect model.

* The labeled TinyOS dataset is loaded from the 'Tinyos_Evaluation_Labled_Multi-Class.csv' file, containing code snippets along with their corresponding ground truth labels. This dataset serves as the basis for evaluating the model's accuracy and other performance metrics.

In [ ]:
#DT Prediction Model Accuracy
# Reading the TinyOS labeled Dataset
TinyOS_labeled_Dataset = pd.read_csv('Tinyos_Evaluation_Labled_Multi-Class.csv',encoding = 'unicode_escape')
TinyOS_Code_Snippet, TinyOS_Code_Tag = TinyOS_labeled_Dataset.code, TinyOS_labeled_Dataset.isMalicious

def preprocess(TinyOS_Code_Snippet):
    return pd.Series(TinyOS_Code_Snippet).replace(r'\b([A-Za-z])\1+\b', '', regex=True)\
        .replace(r'\b[A-Za-z]\b', '', regex=True)

transformer = FunctionTransformer(preprocess)
token_pattern = r"""([A-Za-z_]\w*\b|[!\#\$%\&\*\+:\-\./<=>\?@\\\^_\|\~]+|[ \t\(\),;\{\}\[\]"'`])"""

vectorizer = TfidfVectorizer(token_pattern=token_pattern, max_features=3000)

## Accuracy:
* The proportion of correctly predicted labels compared to the total number of samples.

In [ ]:
# Calculating Accuracy for iDetect
print(f'Accuracy: {pipe_DT.score(TinyOS_Code_Snippet, TinyOS_Code_Tag)}')

Accuracy: 0.7888198757763976


## Weighted F1-Score:
* An average of the F1-Scores for each class, considering the number of samples for each class

In [ ]:
# Calculating Weighted F1-Score for iDetect
f1 = f1_score (TinyOS_Code_Tag, pred,average = 'weighted' )
print('Weighted F1-score of iDetect: %f' % f1)

Weighted F1-score of iDetect: 0.793349


##Macro F1-Score:
* An average of the F1-Scores for each class without considering class imbalance.

In [ ]:
# Calculating Macro  F1-Score for iDetect
f1 = f1_score (TinyOS_Code_Tag, pred,average = 'macro' )
print('Macro F1 score of iDetect: %f' % f1)

Macro F1 score of iDetect: 0.612474


#Classification Report:
* A detailed report showing precision, recall, and F1-Score for each class.

In [ ]:
# Printing Classification Report for iDetect
print(classification_report(TinyOS_Code_Tag, pred))

              precision    recall  f1-score   support

 Benign Code       0.85      0.96      0.90        48
     CWE-119       0.73      0.73      0.73        15
     CWE-120       0.97      0.97      0.97        29
     CWE-126       0.74      1.00      0.85        14
     CWE-134       0.93      0.93      0.93        14
     CWE-190       1.00      1.00      1.00         5
      CWE-20       0.73      0.73      0.73        15
     CWE-327       0.80      1.00      0.89         4
     CWE-350       1.00      1.00      1.00         4
     CWE-362       1.00      0.80      0.89         5
     CWE-367       1.00      0.75      0.86         4
     CWE-377       0.00      0.00      0.00         0
     CWE-398       0.93      0.70      0.80        37
     CWE-401       0.00      0.00      0.00         1
     CWE-457       0.14      0.10      0.12        10
     CWE-467       0.60      0.75      0.67         4
     CWE-476       1.00      0.56      0.71         9
     CWE-477       0.00    

#INSIGHT

* The model performs exceptionally well on classes such as Benign Code, CWE-190, CWE-20, CWE-327, CWE-350, CWE-362, CWE-367, CWE-467, CWE-563, CWE-664, CWE-676, CWE-682, CWE-687, CWE-704, CWE-768, CWE-78, CWE-783, and CWE-807, achieving perfect precision, recall, and F1-scores.

* Classes like CWE-398, CWE-457, CWE-476, CWE-561, and CWE-571 show a balance between precision and recall, resulting in relatively high F1-scores.

* Some classes with limited support exhibit challenges for the model, as indicated by lower precision, recall, and F1-scores. The class CWE-401 has zero precision, indicating potential areas for improvement in model generalization.